In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
from openai import OpenAI

max_tokens = 100

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai.api_key,
)

def get_completion_from_messages(system_message, user_message, delimiter):
    messages = [  
        {'role':'system', 
         'content': system_message},    
        {'role':'user', 
         'content': f"{delimiter}{user_message}{delimiter}"},  
        ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=max_tokens
    )
    return response.choices[0].message.content.strip()

## Initial System Setup

In [3]:
delimiter = "####"

system_message = f"""
You will be provided with a list of patient reviews regarding medications.  
Each review will be delimited with {delimiter} characters.  
Your task is to perform pharmacovigilance by classifying each review into a primary category and a secondary category based on the type of adverse drug reaction (ADR) mentioned.  

Primary categories:  
- Mild Reaction: Reactions that are not life-threatening and have minor effects on the patient's health.  
- Moderate Reaction: Reactions that are more significant but do not require immediate intervention.  
- Severe Reaction: Reactions that are life-threatening or require immediate medical intervention.

Mild Reaction secondary categories:  
- Skin rash  
- Headache  
- Fatigue  
- Gastrointestinal discomfort  

Moderate Reaction secondary categories:  
- Dizziness  
- Nausea  
- Vision changes  
- Breathing difficulty  

Severe Reaction secondary categories:  
- Anaphylaxis  
- Heart attack  
- Organ failure  
- Seizures

Provide the output in JSON format with the keys: "primary" and "secondary."
"""

In [4]:
# Example user review message
user_message = f"""I took the medication and started feeling dizzy along with some nausea after a few hours."""

# Call the completion function
response = get_completion_from_messages(system_message, user_message, delimiter)

# Print the response (classification)
print(response)

{
  "primary": "Moderate Reaction",
  "secondary": [
    "Dizziness",
    "Nausea"
  ]
}


In [5]:
delimiter = "####"

system_message_human_friendly = f"""
You will be provided with a list of patient reviews regarding medications.  
Each review will be delimited with {delimiter} characters.  
Your task is to perform pharmacovigilance by classifying each review into a primary category and a secondary category based on the type of adverse drug reaction (ADR) mentioned.  

Primary categories:  
- Mild Reaction: Reactions that are not life-threatening and have minor effects on the patient's health.  
- Moderate Reaction: Reactions that are more significant but do not require immediate intervention.  
- Severe Reaction: Reactions that are life-threatening or require immediate medical intervention.

Mild Reaction secondary categories:  
- Skin rash  
- Headache  
- Fatigue  
- Gastrointestinal discomfort  

Moderate Reaction secondary categories:  
- Dizziness  
- Nausea  
- Vision changes  
- Breathing difficulty  

Severe Reaction secondary categories:  
- Anaphylaxis  
- Heart attack  
- Organ failure  
- Seizures

Then write a brief summary in which you synthetize the review, explaining it to a medical professional.

Provide the output in JSON format with the keys: "primary", "secondary", "review".
"""

In [6]:
# Example user review message
user_message = f"""I took the medication and started feeling dizzy along with some nausea after a few hours."""

# Call the completion function
response = get_completion_from_messages(system_message_human_friendly, user_message, delimiter)

# Print the response (classification)
print(response)

{
    "primary": "Moderate Reaction",
    "secondary": "Dizziness",
    "review": "The patient reported experiencing dizziness and nausea shortly after taking the medication. These symptoms indicate a moderate reaction to the drug, which may require further monitoring."
}


In [7]:
negative_user_reviews = [
    "After taking the medication, I experienced severe nausea and dizziness.",
    "I developed a rash on my arms and legs after three days of using the drug.",
    "The medication caused constant fatigue, and I felt very weak throughout the day.",
    "I had trouble breathing and felt chest tightness after the first dose.",
    "I noticed my vision became blurry and I felt lightheaded for several hours.",
    "The drug gave me severe headaches, and I couldn’t focus on anything.",
    "After a few weeks, I started experiencing gastrointestinal discomfort and stomach cramps.",
    "I had an allergic reaction, including swelling and difficulty breathing. I had to go to the emergency room.",
    "My heart started racing, and I felt anxious after taking the medication for two days.",
    "I had seizures after starting this medication, which had never happened before."
]

positive_user_reviews = [
    "This medication worked wonders for my migraines, and I felt relief within hours!",
    "After starting this treatment, my energy levels improved, and I no longer feel tired throughout the day.",
    "My blood pressure has been stable ever since I began taking this medication. I’m very satisfied with the results.",
    "This drug completely cleared up my skin condition, and I haven’t had any side effects.",
    "I’ve been using this medication for months, and it’s been a life-changer. My pain has significantly decreased.",
    "The treatment helped me sleep better at night, and I wake up feeling refreshed and full of energy.",
    "My mood has improved dramatically since starting this medication, and I feel much more positive overall.",
    "I haven’t had any side effects, and my symptoms have almost completely disappeared. This drug has been a game-changer.",
    "My joint pain has reduced significantly, and I can now move freely without discomfort.",
    "The medication was easy to take, and I saw improvements within the first week. Very happy with the results!"
]

In [8]:
def evaluate(user_message):
    # Delimiter for the review
    delimiter = "####"
    
    # Call the completion function
    response = get_completion_from_messages(system_message_human_friendly, user_message, delimiter)

    # Print the response (classification)
    # print(response)
    
    return response

In [9]:
# Initialize an empty list to hold combined data
combined_responses = []

# Iterate over negative user reviews and their evaluated responses
for rev in negative_user_reviews:
    response = evaluate(rev)  # Assume this function evaluates the review
    combined_responses.append({"review": rev, "response": response})

# Print the first negative user review and its response
print(combined_responses[0]['review'])
print(combined_responses[0]['response'])

After taking the medication, I experienced severe nausea and dizziness.
{
    "primary": "Moderate Reaction",
    "secondary": [
        "Nausea",
        "Dizziness"
    ],
    "review": "The patient reported experiencing severe nausea and dizziness after taking the medication. These symptoms indicate a moderate adverse drug reaction and may require further evaluation."
}


In [10]:
import json

# Save the combined responses to a JSON file
with open('combined_responses_neg.json', 'w') as file:
    json.dump(combined_responses, file, indent=4)

print("Combined responses have been saved to 'combined_responses.json'.")


Combined responses have been saved to 'combined_responses.json'.


In [11]:
# Initialize an empty list to hold combined data
combined_responses = []

# Iterate over negative user reviews and their evaluated responses
for rev in positive_user_reviews:
    response = evaluate(rev)  # Assume this function evaluates the review
    combined_responses.append({"review": rev, "response": response})

# Print the first negative user review and its response
print(combined_responses[0]['review'])
print(combined_responses[0]['response'])

# Save the combined responses to a JSON file
with open('combined_responses_pos.json', 'w') as file:
    json.dump(combined_responses, file, indent=4)

print("Combined responses have been saved to 'combined_responses_pos.json'.")

This medication worked wonders for my migraines, and I felt relief within hours!
{
    "primary": "Mild Reaction",
    "secondary": "Headache",
    "review": "This medication provided quick relief for migraines with no adverse effects reported."
}
Combined responses have been saved to 'combined_responses_pos.json'.


## Real world Vantin review

In [12]:
def perform_review_analysis(reviews_list, output_filename):
    for rev in reviews_list:
        response = evaluate(rev)  # Assume this function evaluates the review
        combined_responses.append({"review": rev, "response": response})

    # Save the combined responses to a JSON file
    with open(output_filename, 'w') as file:
        json.dump(combined_responses, file, indent=4)

In [13]:
vantin_user_reviews = []
vantin_user_reviews_link = ''

with open('Vantin_user_reviews.txt', 'r') as f:
    for line in f:
        if 'http' in line:
            vantin_user_reviews_link = line
        else:
            vantin_user_reviews.append(line)

In [14]:
print(vantin_user_reviews[0])

For Pneumonia "On my fourth pill of Vantin in a few more hours, and I couldn't believe the rapidity of the healing! I am taking it for Klebsiella pneumoniae, along with Macrobid for Enterobacter bacteria! I have had dry mouth, dry throat, increased thirst, slight ringing in ears that has passed, headache, ugh, was the worst so far, but not horrible. Heavier menstrual period, a little dizziness, and hey! I'm penicillin allergic. So I think I'm doing A-1."



In [15]:
filename = 'vantin_review_analysis_result.json'
perform_review_analysis(vantin_user_reviews, filename)

## Prettify json output

In [16]:
with open(filename, 'r') as file:
    data = json.load(file)

print(data)

[{'review': 'This medication worked wonders for my migraines, and I felt relief within hours!', 'response': '{\n    "primary": "Mild Reaction",\n    "secondary": "Headache",\n    "review": "This medication provided quick relief for migraines with no adverse effects reported."\n}'}, {'review': 'After starting this treatment, my energy levels improved, and I no longer feel tired throughout the day.', 'response': '{\n    "primary": "Mild Reaction",\n    "secondary": "Fatigue",\n    "review": "After starting this treatment, the patient experienced improved energy levels and no longer feels tired throughout the day."\n}'}, {'review': 'My blood pressure has been stable ever since I began taking this medication. I’m very satisfied with the results.', 'response': '{\n    "primary": "Mild Reaction",\n    "secondary": null,\n    "review": "The patient experienced stable blood pressure and is satisfied with the medication."\n}'}, {'review': 'This drug completely cleared up my skin condition, and 

In [17]:
import pandas as pd
from IPython.display import HTML

# Convert JSON to DataFrame
df = pd.DataFrame(data)

# Create an HTML table
html_table = df.to_html(classes='table table-striped', index=False)

# Display the HTML
HTML(html_table)


C:\Users\Matteo\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


review,response
"This medication worked wonders for my migraines, and I felt relief within hours!","{\n ""primary"": ""Mild Reaction"",\n ""secondary"": ""Headache"",\n ""review"": ""This medication provided quick relief for migraines with no adverse effects reported.""\n}"
"After starting this treatment, my energy levels improved, and I no longer feel tired throughout the day.","{\n ""primary"": ""Mild Reaction"",\n ""secondary"": ""Fatigue"",\n ""review"": ""After starting this treatment, the patient experienced improved energy levels and no longer feels tired throughout the day.""\n}"
My blood pressure has been stable ever since I began taking this medication. I’m very satisfied with the results.,"{\n ""primary"": ""Mild Reaction"",\n ""secondary"": null,\n ""review"": ""The patient experienced stable blood pressure and is satisfied with the medication.""\n}"
"This drug completely cleared up my skin condition, and I haven’t had any side effects.","{\n ""primary"": ""Mild Reaction"",\n ""secondary"": ""Skin rash"",\n ""review"": ""The patient experienced a mild reaction to the medication, with the primary benefit of the drug being the clearance of their skin condition. No side effects were reported.""\n}"
"I’ve been using this medication for months, and it’s been a life-changer. My pain has significantly decreased.","{\n ""primary"": ""Mild Reaction"",\n ""secondary"": ""N/A"",\n ""review"": ""The patient experienced a significant decrease in pain after using the medication for months, indicating a mild reaction.""\n}"
"The treatment helped me sleep better at night, and I wake up feeling refreshed and full of energy.","{\n ""primary"": ""Mild Reaction"",\n ""secondary"": ""Fatigue"",\n ""review"": ""The patient reported feeling refreshed and full of energy after using the treatment for better sleep at night. This indicates a mild reaction related to improved energy levels.""\n}"
"My mood has improved dramatically since starting this medication, and I feel much more positive overall.","{\n ""primary"": ""Mild Reaction"",\n ""secondary"": ""None"",\n ""review"": ""The patient experienced improved mood and overall positivity since starting the medication. No adverse drug reactions were mentioned.""\n}"
"I haven’t had any side effects, and my symptoms have almost completely disappeared. This drug has been a game-changer.","{\n ""primary"": ""Mild Reaction"",\n ""secondary"": ""N/A"",\n ""review"": ""The patient reported no side effects and a significant improvement in symptoms after taking the medication, considering it a game-changer.""\n}"
"My joint pain has reduced significantly, and I can now move freely without discomfort.","{\n ""primary"": ""Mild Reaction"",\n ""secondary"": ""None"",\n ""review"": ""The patient experienced a reduction in joint pain, allowing for improved mobility without discomfort. No adverse drug reactions were reported.""\n}"
"The medication was easy to take, and I saw improvements within the first week. Very happy with the results!","{\n ""primary"": ""Mild Reaction"",\n ""secondary"": ""N/A"",\n ""review"": ""The patient experienced improvements within the first week of taking the medication. Overall, the medication was well-tolerated and resulted in a positive outcome.""\n}"


In [22]:
df.head()
df['parsed_response'] = df['response'].apply(json.loads)
normalized_df = pd.json_normalize(df['parsed_response'])

normalized_df.head()

,primary,secondary,review
0,Mild Reaction,Headache,This medication provided quick relief for migr...
1,Mild Reaction,Fatigue,"After starting this treatment, the patient exp..."
2,Mild Reaction,None,The patient experienced stable blood pressure ...
3,Mild Reaction,Skin rash,The patient experienced a mild reaction to the...
4,Mild Reaction,N/A,The patient experienced a significant decrease...


In [27]:
reviews = df['review']  # This column already contains the reviews
final_df = pd.concat([normalized_df, reviews], axis=1)
final_df.columns = ['primary', 'secondary', 'technical_review', 'user_review']

final_df.head()

,primary,secondary,technical_review,user_review
0,Mild Reaction,Headache,This medication provided quick relief for migr...,This medication worked wonders for my migraine...
1,Mild Reaction,Fatigue,"After starting this treatment, the patient exp...","After starting this treatment, my energy level..."
2,Mild Reaction,None,The patient experienced stable blood pressure ...,My blood pressure has been stable ever since I...
3,Mild Reaction,Skin rash,The patient experienced a mild reaction to the...,This drug completely cleared up my skin condit...
4,Mild Reaction,N/A,The patient experienced a significant decrease...,"I’ve been using this medication for months, an..."
